#### Import

In [1]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [3]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10", use_rt=True)
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1418


<Transform:
<Orientation: 
array([[-0.00223963,  0.86520136, -0.50141957],
       [ 0.99961827, -0.01187069, -0.02494779],
       [-0.02753706, -0.50128404, -0.86484451]])>
<Vector: (-0.34373, 0.45093, 0.46060)>
>


In [4]:
from ddh import DDGripper
from scooping_primitives import HighSpeedScooping
gripper = DDGripper("ddh_scooping")
scoop = HighSpeedScooping(ur, gripper, "hss")

reading gripper config...
connecting to odrive...
found left finger
found right fingers
reading scooping config...


#### script testing

In [4]:
gripper.arm()
gripper.set_stiffness(20)
gripper.set_left_tip((158, 38))
gripper.set_right_tip((158, -38))
# gripper.set_parallel_jaw(-14.5, 0)

Setting left tip: (158, 38)
Setting right tip: (158, -38)


#### test deceleration control

In [140]:
ur.speedl([0,0,-0.4,0,0,0],0.4,10)

In [141]:
# print(ur.get_tcp_speed(wait=False)[0])
stop_dist = 0.05
z_spd = ur.get_tcp_speed(wait=False)[2]
stop_dec = (z_spd**2)/(2*stop_dist)
ur.stopl(stop_dec)
pos = ur.getl()[2]

In [142]:
stop_dist = ur.getl()[2] - pos
print(stop_dist)
print(z_spd)

-0.05322568129083427
-0.4001141268684379


#### deceleration control and end pos control

In [91]:
ur.speedl([0.4,0,0,0,0,0],0.4,10)

In [92]:
# print(ur.get_tcp_speed(wait=False)[0])
stop_dist = 0.05
end_dist = 0.3
reverse_vel = 0.4
spd_detect = ur.get_tcp_speed(wait=False)[0]
stop_dec = (spd_detect**2)/(2*stop_dist)
ur.speedl([-reverse_vel,0,0,0,0,0],stop_dec,10)
pos_dec = ur.getl()[0]
while ur.get_tcp_speed(wait=False)[0] > 0:
    continue
pos_stop = ur.getl()[0]
print("Reach zero vel!")
t_acc = reverse_vel / stop_dec
s_acc = 0.5 * stop_dec * t_acc**2
t_const = (end_dist-s_acc) / reverse_vel
print(s_acc)
print(t_acc+t_const)
time.sleep(t_acc+t_const)
ur.stopl(5)
pos_end = ur.getl()[0]

Reach zero vel!
0.049768522491737695
0.8744213062293441


In [93]:
ur.stopl(2)
stop_dist = pos_stop - pos_dec
reverse_dist = pos_end - pos_stop
print(spd_detect)
print(stop_dec)
print(stop_dist)
print(reverse_dist)

0.40092913739670216
1.6074417321366368
0.053455938681710294
-0.3207241538181202


break


In [98]:
pose = ur.get_pose()
print(pose)
print(ur.getl())

<Transform:
<Orientation: 
array([[ 0.00779014,  0.95668964, -0.29100558],
       [ 0.99974421, -0.01363056, -0.01804798],
       [-0.02123288, -0.29079055, -0.95655111]])>
<Vector: (-0.27695, 0.43693, 0.59485)>
>
[-0.2769455458607862, 0.4369331464189327, 0.5948543810792469, -2.0824007304925716, -2.0597256505179344, 0.3287233937577874]


#### scooping test

In [26]:
scoop.initialize_gripper_pose((-0.509,0.45,180))
time.sleep(0.5)
a2_init = gripper.right_a2

Setting left tip: (154.3, 27.55218152745383)
Setting right tip: (134.95, -8.932635346088396)
Setting pose: 
[-0.5176      0.45        0.2        -1.92687451 -1.92687451  0.51630447]


In [24]:
# smack and scoop
smack_vel = 0.3 #100
smack_acc = 0.3 #0.3
stop_smack_acc = 2 #7
reverse_vel = 0.4 # 250
reverse_acc = 0.35 #1
stop_acc = 0.5 #0.5
stop_dist = 0.005

# robot accelerate
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)
# robot detect collision, then stop, close gripper, and go up
while 1:
    a2_cur = gripper.right_a2
    #print(a2_cur)
    if a2_cur - a2_init > 0.3:
        print ("Collision detected!")
        z_spd = ur.get_tcp_speed(wait=False)[2]
        stop_dec = (z_spd**2)/(2*stop_dist)
        gripper.set_stiffness(20, 'R')
        gripper.set_left_tip((158, 38))
        gripper.set_right_tip((158, -38))
        ur.stopl(stop_smack_acc)
        gripper.set_stiffness(100)
#         ur.speedl([0,0,reverse_vel,0,0,0],reverse_acc,5)
        time.sleep(0.5)
        ur.stopl(stop_acc)
        break

Collision detected!
Setting left tip: (158, 38)
Setting right tip: (158, -38)


In [25]:
print(z_spd)
print(stop_dec)

-0.2968776723154525
8.813635231944119


In [ ]:
ur.stopl()